#### **상태 설정**

In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

#### **ToolNode로 도구 노드 구축**

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition

tool = TavilySearchResults(max_results=2)
tools = [tool]
tool_node = ToolNode(tools)

#### **Tool Node가 무엇일까**

In [3]:
import json

from langchain_core.messages import ToolMessage

class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}

#### **LLM 챗봇 설정**

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    result = llm_with_tools.invoke(state["messages"])
    return {"messages": [result]}

#### **그래프 구축**

In [5]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()

**인터넷 검색이 필요한 질문**

In [6]:
graph.invoke({"messages": {"role": "user", "content": "지금 한국 대통령은 누구야?"}})

{'messages': [HumanMessage(content='지금 한국 대통령은 누구야?', additional_kwargs={}, response_metadata={}, id='5dd8f85a-86bb-4fd1-96ef-b1e903c7b86e'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_T70CYAN9vn5scguaD97bMjxw', 'function': {'arguments': '{"query":"현재 한국 대통령"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 87, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-07f9500a-23f2-4e68-ab9d-00ace0923c03-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '현재 한국 대통령'}, 'id': 'call_T70CYAN9vn5scguaD97bMjxw', 'type': 'tool_call'}],

**LLM이 답할 수 있는 질문**

In [7]:
graph.invoke({"messages": {"role": "user", "content": "마이크로소프트가 어떤 회사야?"}})

{'messages': [HumanMessage(content='마이크로소프트가 어떤 회사야?', additional_kwargs={}, response_metadata={}, id='cca5b84c-fb97-4f7c-bf23-1b5afc9e22de'),
  AIMessage(content='마이크로소프트(Microsoft)는 미국의 다국적 기술 기업으로, 1975년에 빌 게이츠와 폴 앨런에 의해 설립되었습니다. 본사는 워싱턴주 레드몬드에 위치하고 있으며, 소프트웨어, 하드웨어, 전자 기기 등 다양한 분야에서 활동하고 있습니다.\n\n마이크로소프트의 가장 잘 알려진 제품에는 운영 체제인 Windows, 생산성 소프트웨어인 Microsoft Office, 클라우드 서비스인 Azure, 그리고 Xbox 게임 콘솔 등이 있습니다. 또한, 최근에는 인공지능, 머신러닝, 데이터 분석 등 첨단 기술 분야에도 많은 투자를 하고 있습니다.\n\n마이크로소프트는 기술 혁신을 통해 개인과 기업에 다양한 솔루션을 제공하며, 전 세계적으로 큰 영향력을 미치고 있는 기업입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 180, 'prompt_tokens': 89, 'total_tokens': 269, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'finish_reason': 'stop', 'logpr